# Gamma higher

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import egttools as egt

In [2]:
# Parameters for the pay-off matrix
## The reward values (prisoner's dilemma)
R_pd = 1
S_pd = -1
T_pd = 2
P_pd = 0
## The reward values (snowdrift)
R_sd = 1
S_sd = 0
T_sd = 2
P_sd = -1
## The values used to calculate the reward for a certain strategy
gamma = 1 # Importance of the game
r = 50 # number of rounds
theta = 3 # trust threshold
p = 0.25 # Chance of checking
epsilon = 0.25 # opportunity cost
# Parameters for the game settings
beta = 0.1 # intensity of selection
Z = 100 # Population size (N)
mu = 0 # the mutation probability
nb_strategies_with = 5 # The amount of strategies
strategies = ["ALLC","ALLD","TFT","TUC","TUD"] # The name of each strategy
colors_strat = ["tab:blue","tab:orange","tab:green","tab:red","tab:purple"] # the color for each strategy
nb_strategies_without = 3 # the amount of strategies without TUC and TUD
colors_trust = ["tab:orange","tab:blue"] # The color for the frequency of trust
# boolean to run smaller experiments seperatly
run_small = True
run_big = True

In [3]:
def calculate_pay_off(R,S,T,P,gamma,r,theta,p,epsilon):
    # Multiply the rewards with the importance of the game
    R *= gamma
    S *= gamma
    T *= gamma
    P *= gamma
    # Calculate the value of each strategy
    # static table
    ## Always cooperate (ALLC)
    ALLC_ALLC = R
    ALLC_ALLD = S
    ALLC_TFT = R
    ALLC_TUC = R
    ALLC_TUD = (theta*R+(r-theta)*S)/r
    ALLC = [ALLC_ALLC,ALLC_ALLD,ALLC_TFT,ALLC_TUC,ALLC_TUD]
    ## Always defect (ALLD)
    ALLD_ALLC = T
    ALLD_ALLD = P
    ALLD_TFT = (T+(r-1)*P)/r
    ALLD_TUC = (T+(r-1)*P)/r
    ALLD_TUD = (T+(r-1)*P)/r
    ALLD = [ALLD_ALLC,ALLD_ALLD,ALLD_TFT,ALLD_TUC,ALLD_TUD]
    ## Tit for Tat (TFT)
    TFT_ALLC = R-epsilon
    TFT_ALLD = (S+(r-1)*P)/r - epsilon
    TFT_TFT = R - epsilon
    TFT_TUD = R - epsilon
    TFT_TUC = (theta*R+S+(r-theta-1)*P)/r - epsilon
    TFT = [TFT_ALLC,TFT_ALLD,TFT_TFT,TFT_TUD,TFT_TUC]
    ## Trust-based Cooperate (TUC)
    TUC_ALLC = R - (theta*epsilon)/r - (p*(r-theta)*epsilon)/r
    TUC_ALLD = (S + (r-1)*P)/r - epsilon
    TUC_TFT = R-(theta*epsilon)/r-(p*(r-theta)*epsilon)/r
    TUC_TUC = R-(theta*epsilon)/r-(p*(r-theta)*epsilon)/r
    TUC_TUD = (theta*R-theta*epsilon)/r + (1/r)*((S*(1-(1-p)**(r-theta)))/p + ((P-epsilon)*((1-p)**(r-theta)+(r-theta)*p - 1))/p)
    TUC = [TUC_ALLC,TUC_ALLD,TUC_TFT,TUC_TUC,TUC_TUD]
    ## Trust-based Defect (TUD)
    TUD_ALLC = (epsilon*R+(r-epsilon)*T-theta*epsilon)/r
    TUD_ALLD = (S+(r-1)*P)/r-epsilon
    TUD_TFT = (theta*R+T+(r-theta-1)*P-theta*epsilon)/r
    TUD_TUC = (theta*R-theta*epsilon)/r + (1/r)*((T*(1 - (1 - p) ** (r - theta)))/p + (P*((1-p)**(r-theta) + (r - theta)*p - 1))/p)
    TUD_TUD = (theta*R+(r - theta) * P - theta*epsilon)/r
    TUD = [TUD_ALLC,TUD_ALLD,TUD_TFT,TUD_TUC,TUD_TUD]
    # Return the values as an array
    return np.array([
        ALLC,
        ALLD,
        TFT,
        TUC,
        TUD,])

In [4]:
# The function for testing the influence of gamma and r
def test_gamma_r(R,S,T,P,theta,p,epsilon,nb_strategies_with,nb_strategies_without,beta,Z,mu,gammas,rs):
    # array for the results with the trust-strategies and without the trust-strategies
    results = []
    results_without_r = []
    results_with_r = []
    # go over every r in the rs
    for r in rs:
        # for the frequencies of strategies
        results_with = np.zeros((nb_strategies_with,len(gammas)))
        # for the frequency of cooperation
        results_without = np.zeros((nb_strategies_without,len(gammas)))
        # go over all the gammas
        for pos in range(len(gammas)):
            # get the gamma
            gamma = gammas[pos]
            # calculate the pay-offs
            pay_off = calculate_pay_off(R,S,T,P,gamma,r,theta,p,epsilon)
            # for all strategies
            evolver = egt.analytical.StochDynamics(nb_strategies_with,pay_off,Z,mu)
            # calculate the stationary distribution
            stationary_SML = evolver.calculate_stationary_distribution(beta)
            # store the results
            for strategy,v in zip(range(nb_strategies_with),stationary_SML):
                results_with[strategy][pos] = v
            # for all strategies except TUC and TUD
            evolver = egt.analytical.StochDynamics(nb_strategies_without, pay_off, Z,mu)
            # calculate the stationary distribution
            stationary_SML = evolver.calculate_stationary_distribution(beta)
            # safe the result
            for strategy,v in zip(range(nb_strategies_without),stationary_SML):
                results_without[strategy][pos] = v
        # append the results to the global result collection
        results.append(results_with)
        results_without_r.append(np.array([results_without[0][pos] + results_without[2][pos] for pos in range(len(gammas))]))
        results_with_r.append(np.array([results_with[0][pos] + results_with[2][pos] + results_with[3][pos] + (results_with[4][pos] * (theta/r)) for pos in range(len(gammas))]))
    return results,results_without_r,results_with_r

# This function visualises the results
def visualise_gamma_r(ax_strats,ax_coops,nb_strategies_with,strategies,colors_strat,nb_strategies_without,colors_trust,gammas,rs,results,results_without_trust,results_with_trust):
    for r,ax_strat,ax_coop,result,without_trust,with_trust in zip(rs,ax_strats,ax_coops,results,results_without_trust,results_with_trust):
        # for the frequencies of strategies
        # Plot the line for each of the strategies
        for pos,color_strat in zip(range(nb_strategies_with),colors_strat):
            ax_strat.plot(gammas,result[pos],label = strategies[pos],color = color_strat)
        # enable the legend for the graph
        ax_strat.legend()
        # give the x and y axis a label
        ax_strat.set_xlabel("importance of the game $\gamma$")
        ax_strat.set_ylabel("Frequency of Strategy")
        # put the x axis to a exponential scale
        ax_strat.set_xscale("symlog")
        # limit the x and y axis
        ax_strat.set_xlim(0.1,6900)
        ax_strat.set_ylim(-0.1,1.1)
        # for the frequency of cooperation
        # Get the values of all the strategies that like to cooperate (without trust)
        ax_coop.plot(gammas,without_trust,label = "without TUC/TUD",color = colors_trust[0])
        # Get the values of all the strategies that like to cooperate (with trust)
        ax_coop.plot(gammas,with_trust,label = "with TUC/TUD",color = colors_trust[1])
        # color the gaps in between
        ax_coop.fill_between(gammas, with_trust, without_trust, where=(with_trust > without_trust), color='green', alpha=0.3)
        ax_coop.fill_between(gammas, with_trust, without_trust, where=(with_trust < without_trust), color='red', alpha=0.3)
        # enable the legend
        ax_coop.legend()
        # give the x and y axis a label
        ax_coop.set_xlabel("importance of the game $\gamma$")
        ax_coop.set_ylabel("Frequency of Cooperation")
        # put the x axis to an exponential scale
        ax_coop.set_xscale("symlog")
        # limit the x and y axis
        ax_coop.set_xlim(0.1,6900)
        ax_coop.set_ylim(-0.1,1.1)
        # add titles
        ax_strat.set_title(f"r = {r}",size = 15)

In [5]:
# all the gammas and rs we want to test
gammas = np.geomspace(0.1,6900,100)
rs = np.array([20,50,100])

#### Prisoner's dilemma

In [ ]:
figs = []
for theta in [3,5,10]:
    # run the actual experiment
    gammas_result,gammas_without_r,gammas_with_r = test_gamma_r(R_pd,S_pd,T_pd,P_pd,theta,p,epsilon,nb_strategies_with,nb_strategies_without,beta,Z,mu,gammas,rs)
    # create a subplot for the results
    fig_gamma,(ax_strats_gamma,ax_coops_gamma) = plt.subplots(2,3,figsize=(18,10))
    # add a title
    fig_gamma.suptitle("Influence of $\gamma$ and r where " + r"$\theta$ = " + f"{theta}",size=20)
    # visualise the results
    visualise_gamma_r(ax_strats_gamma,ax_coops_gamma,nb_strategies_with,strategies,colors_strat,nb_strategies_without,colors_trust,gammas,rs,gammas_result,gammas_without_r,gammas_with_r)
    # save a figure of the results
    fig_gamma.savefig(f'figure/idp_bigger_gamma_{theta}.png')
    figs.append(fig_gamma)

In [ ]:
for fig in figs:
    plt.close(fig)